# Trabalho Prático Final 

Sistema de recomendação de games

In [1]:
# Instalando a lib surprise
!pip install surprise

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd


games = pd.read_csv('games.csv')
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
games.columns = ['id_game','titulo','genero']


#Buscando games assistidos e não assistidos de um usuário

In [3]:
games_jogados = notas.query('id_usuario == 1')
games_nao_jogados = games.set_index('id_game').drop(games_jogados.set_index('id_game').index,errors='ignore')
games_nao_jogados.head(5)

,titulo,genero
id_game,,
5,Minecraft,Sandbox
12,Halo: Combat Evolved,Shooters (FPS and TPS)
13,The Last of Us,Survival and horror
14,GoldenEye 007,Shooters (FPS and TPS)
19,Grand Theft Auto: San Andreas,Sandbox


In [4]:
# Conferindo os games não jogados
# Ele compara o id do usuário e o id do game e se não retornar nada, é porque está correto.
games_jogados = notas.query('id_usuario == 584 and id_game == 1237')
games_jogados

,id_usuario,id_game,nota_usuario_game,dia_avaliacao


#Criando, treinando, salvando e carregando modelo

In [5]:
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split 
import pickle

In [6]:
# Reader = escala das notas, a estrutura da linha de entrada de dados -> id_u id_i avaliacao
reader = Reader(line_format='user item rating',rating_scale=(0.0,5.0))

In [7]:
# Preparando os dados para a lib surprise, que deve ter somente as colunas id do usuario, id do game e a nota
dados_surprise = notas[['id_usuario','id_game','nota_usuario_game']]
dados_preparados_treinamento = surprise.Dataset.load_from_df(dados_surprise,reader)

In [8]:
# Temos os dados preparados em formato objeto surprise
# Criando o modelo SVD
algo = surprise.SVDpp()

In [9]:
trainset, testset = train_test_split(dados_preparados_treinamento)

In [10]:
# Treinando o modelo
algo.fit(trainset)

In [11]:
pickle.dump(algo,open('treinado.pkl','wb'))

In [12]:
print('Predições dos games não jogados do usuário 1')
dic=[]
#vamos pegar todos os games não jogados, calcular a predição e salvar no dicionario
games_nao_jogados = games_nao_jogados.reset_index()
for index, row in games_nao_jogados.iterrows():
  dic.append(algo.predict(1,row['id_game']))
  

Predições dos games não jogados do usuário 1


In [13]:
predicoes = pd.DataFrame.from_dict(dic)

In [14]:
predicoes.columns = ['id_usuario','id_game','ref','predicao','destalhes']

In [15]:
recomendacoes = pd.merge(predicoes,games,how='inner',on='id_game').sort_values('predicao',ascending=False)[['id_game','titulo','predicao']]

#Modulando funções para buscar games jogados e não jogados pelo id_game

In [16]:
def busca_games_jogados(id_usuario):
  print('Games jogados do usuário %d' % id_usuario)
  games_jogados = notas.query('id_usuario == %d' % id_usuario)
  return games_jogados

In [17]:
def busca_games_nao_jogados(id_usuario):
  games_jogados = notas.query('id_usuario == %d' % id_usuario)
  id_games_jogados = games_jogados.set_index('id_game').index
  games_nao_jogados = games.set_index('id_game').drop(id_games_jogados,errors='ignore').reset_index()
  return games_nao_jogados


In [18]:
def verifica_predicoes_games_nao_jogados(id_usuario):
  print('Predicoes do usuário %d' % id_usuario)
  dic=[]
  # vamos pegar todos os games não jogados, calcular a predição e salvar no dicionario
  games_predicoes = busca_games_nao_jogados(id_usuario)
  for index, row in games_predicoes.iterrows():
    dic.append(algo.predict(id_usuario,row['id_game']))
  return pd.DataFrame.from_dict(dic)

# Realizando predições

In [19]:
def apresenta_recomendacoes(id_usuario,numero_de_recomendacoes=10):
  games_jogados = notas.query("id_usuario == %d " % id_usuario)
  if games_jogados.empty:
    print("****O usuário com id %d ainda não avaliou nenhum game****" % id_usuario)
    print("****Serão recomendados os games mais populares****")
    total_de_votos = notas['id_game'].value_counts()
    games_backup = games.copy()
    games_backup['total_de_votos'] = total_de_votos
    notas_medias = notas.groupby('id_game').mean()['nota_usuario_game']
    games_backup['notas_medias'] = notas_medias
    return games_backup.sort_values('total_de_votos',ascending=False).head(numero_de_recomendacoes)

  predicoes = verifica_predicoes_games_nao_jogados(id_usuario)
  predicoes.columns = ['id_usuario','id_game','ref','predicao','destalhes']
  recomendacoes = pd.merge(predicoes,games,how='inner',on='id_game').sort_values('predicao',ascending=False)[['id_game','titulo','predicao']]
  return recomendacoes.head(numero_de_recomendacoes)

In [20]:
apresenta_recomendacoes(950,5)

****O usuário com id 950 ainda não avaliou nenhum game****
****Serão recomendados os games mais populares****


,id_game,titulo,genero,total_de_votos,notas_medias
34,35,Dark Souls,Role-playing,12.0,4.666667
12,13,The Last of Us,Survival and horror,12.0,4.250000
5,6,Grand Theft Auto V,Sandbox,11.0,4.454545
23,24,Final Fantasy VII,Role-playing,10.0,4.600000
4,5,Minecraft,Sandbox,10.0,4.100000


In [21]:
apresenta_recomendacoes(584)

Predicoes do usuário 584


,id_game,titulo,predicao
33,34,Chrono Trigger,4.768903
21,22,Super Mario Bros. 3,4.751032
47,48,The Sims,4.725624
14,15,The Legend of Zelda: Breath of the Wild,4.698387
27,28,Shadow of the Colossus,4.685214
37,38,Grand Theft Auto: Vice City,4.660263
46,47,Portal,4.653465
12,13,The Last of Us,4.650583
48,49,Call of Duty 4: Modern Warfare,4.644270
15,16,Doom,4.628062


#Inserindo dados no arquivo CSV

In [22]:
import csv

In [23]:
with open('ratings.csv','a') as arquivo_csv:
  escrever = csv.writer(arquivo_csv,delimiter=',',lineterminator='\n')
  escrever.writerow([584,1223,5.0])

In [24]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']

In [25]:
notas.tail()

,id_usuario,id_game,nota_usuario_game,dia_avaliacao
199,584,1223,5.0,NaN
200,584,1223,5.0,NaN
201,584,1223,5.0,NaN
202,584,1223,5.0,NaN
203,584,1223,5.0,NaN


In [26]:
notas.tail()

,id_usuario,id_game,nota_usuario_game,dia_avaliacao
199,584,1223,5.0,NaN
200,584,1223,5.0,NaN
201,584,1223,5.0,NaN
202,584,1223,5.0,NaN
203,584,1223,5.0,NaN


#Salvar predições em arquivo xml

In [27]:
import xml.etree.cElementTree as ET

In [28]:
def grava_arquivo_xml(dados,id_usuario):
    root = ET.Element("Catalogo")
    for index, row in dados.iterrows():
        doc = ET.SubElement(root, "Game")
        ET.SubElement(doc, "Id_game").text = str(row['id_game'])
        ET.SubElement(doc, "Titulo").text = row['titulo']
    tree = ET.ElementTree(root)
    tree.write("games_us_"+str(id_usuario)+".xml")


In [29]:
grava_arquivo_xml(apresenta_recomendacoes(145,5),145)

****O usuário com id 145 ainda não avaliou nenhum game****
****Serão recomendados os games mais populares****


#Função para realizar treinamento

In [30]:
#essa função vai criar o modelo e treinamento com os dados.
#escala menor é a nota mais baixa do conjunto de dados. -> 0.0
#escala maior é a nota mais alta do conjunto de dados. -> 0.5
def realiza_treinamento(dados, nome_modelo,escala_menor, escala_maior):
  if(nome_modelo == 'SVD') or (nome_modelo == 'svd'):
    print('Modelo escolhido foi o SVD')
    algo = surprise.SVD()
    print('Modelo gerado com sucesso')
  elif(nome_modelo == 'SVDpp') or (nome_modelo == 'svdpp') or (nome_modelo == 'SVDPP'):
    print('Modelo escolhido foi o SVDpp')
    algo = surprise.SVDpp()
    print('Modelo gerado com sucesso')

  reader = Reader(line_format='user item rating',rating_scale=(escala_menor,escala_maior))
  dados_surprise = dados[['id_usuario','id_game','nota_usuario_game']]
  dados_preparados_treinamento = surprise.Dataset.load_from_df(dados_surprise,reader)
  trainset, testset = train_test_split(dados_preparados_treinamento)
  algo.fit(trainset)
  print("Modelo "+nome_modelo+" treinado com sucesso")
  return algo


In [31]:
algo = realiza_treinamento(notas,'SVD',0.0,5.0)

Modelo escolhido foi o SVD
Modelo gerado com sucesso
Modelo SVD treinado com sucesso


#Criando menu para inserir avaliações, recarregar treinamento, apresentar recomendações

In [32]:
#função para salvar o modelo treinado
def salva_modelo(algo):
  print(algo)
  pickle.dump(algo,open('treinado.pkl','wb'))
  print("O comando dump foi executado sem erro")

In [33]:
def carrega_modelo():
  with open('treinado.pkl','rb') as modelo:
    algo = pickle.load(modelo)
  return algo

In [34]:
def verifica_predicoes_games_nao_jogados(id_usuario,algo):
  print('Predicoes do usuário %d' % id_usuario)
  dic=[]
  #vamos pegar todos os games não jogados, calcular a predição e salvar no dicionario
  games_predicoes = busca_games_nao_jogados(id_usuario)
  for index, row in games_predicoes.iterrows():
    dic.append(algo.predict(id_usuario,row['id_game']))
  return pd.DataFrame.from_dict(dic)

In [35]:
def apresenta_recomendacoes(id_usuario,notas,algo,games,numero_de_recomendacoes=10):
  games_assistidos = notas.query("id_usuario == %d " % id_usuario)
  if games_assistidos.empty:
    print("****O usuário com id %d ainda não avaliou nenhum game****" % id_usuario)
    print("****Serão recomendados os games mais populares****")
    total_de_votos = notas['id_game'].value_counts()
    games_backup = games.copy()
    games_backup['total_de_votos'] = total_de_votos
    notas_medias = notas.groupby('id_game').mean()['nota_usuario_game']
    games_backup['notas_medias'] = notas_medias
    return games_backup.sort_values('total_de_votos',ascending=False).head(numero_de_recomendacoes)

  predicoes = verifica_predicoes_games_nao_jogados(id_usuario,algo)
  predicoes.columns = ['id_usuario','id_game','ref','predicao','destalhes']
  recomendacoes = pd.merge(predicoes,games,how='inner',on='id_game').sort_values('predicao',ascending=False)[['id_game','titulo','predicao']]
  return recomendacoes.head(numero_de_recomendacoes)

In [36]:
def menu():
  continuar = 1
  while(continuar):
    continuar = int(input("0. Sair\n"+
                          "1. Inserir novas avaliações\n"+
                          "2. Verificar id de um novo usuário\n"+
                          "3. Realiza Treinamento\n"+
                          "4. Recomenda Games\n"
                          )
                    )
    if(continuar == 1):
      usuario = input("Digite o id do usuário: ")
      game = input("Digite o id do game: ")
      nota = input("Digite a nota do game: ")

      with open('ratings.csv','a') as arquivo_csv:
        escrever = csv.writer(arquivo_csv,delimiter=',',lineterminator='\n')
        escrever.writerow([usuario,game,nota])

    if(continuar == 2):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
      novo_usuario = notas['id_usuario'].max()+1
      print("O próximo ID de usuário livre eh: %d" % novo_usuario)

    if(continuar == 3):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
      modelo = input("Digite o nome do modelo: ")
      escala_menor = float(input("Digite o valor da nota mais baixa: "))
      escala_maior = float(input("Digite o valor da nota mais alta: "))
      algo = realiza_treinamento(notas,modelo,escala_menor,escala_maior)
      salva_modelo(algo)

    if(continuar == 4):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
      algo = carrega_modelo()
      id_usuario = int(input("Recomendar games para: "))
      numero_de_recomendacoes = int(input("Digite o número de games a recomendar: "))
      recomendacoes = apresenta_recomendacoes(id_usuario,notas,algo,games,numero_de_recomendacoes)
      grava_arquivo_xml(recomendacoes,id_usuario)
      print(recomendacoes[['id_game','titulo']])
      

In [37]:
menu()

In [38]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
notas.tail(5)

,id_usuario,id_game,nota_usuario_game,dia_avaliacao
199,584,1223,5.0,NaN
200,584,1223,5.0,NaN
201,584,1223,5.0,NaN
202,584,1223,5.0,NaN
203,584,1223,5.0,NaN


#Verificar recomendações após novas avaliações

#Base de dados escolhida

https://github.com/andredarcie/best-games-of-all-time-data-based/blob/master/dataset/generated_game_list.csv